### Librerías

In [14]:
import cv2
import torch
import easyocr
import pytesseract
import pandas as pd
from ultralytics import YOLO
import time

### Vídeo

In [ ]:
# CONFIGURACIÓN
video_input = "test.mp4"           # Ruta del video de entrada
video_output = "resultado.mp4"     # Video con resultados
csv_output = "resultados.csv"      # CSV de salida

# Ruta al ejecutable de Tesseract
pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract.exe'

# Inicializar OCR (inglés → mejor para texto alfanumérico)
reader_easy = easyocr.Reader(['en'], gpu=True)

# Cargar modelos YOLO
modelo_vehiculos = YOLO("yolo11n.pt").to('cuda')
modelo_matriculas = YOLO("best.pt").to('cuda')

# CONFIGURAR VIDEO
cap = cv2.VideoCapture(video_input)
if not cap.isOpened():
    print("Error: no se puede abrir el video")
    exit()

fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(video_output, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

resultados = []
frame_id = 0

# PROCESAMIENTO DE FRAMES
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Detección y tracking de vehículos
    pred = modelo_vehiculos.track(frame, persist=True)[0]

    for obj in pred.boxes:
        x1, y1, x2, y2 = map(int, obj.xyxy[0])
        tipo = modelo_vehiculos.names[int(obj.cls[0])]
        conf = float(obj.conf[0])
        track_id = int(obj.id[0]) if hasattr(obj, 'id') else -1

        matricula_easy = ""
        matricula_tesseract = ""
        time_easy = 0
        time_tesseract = 0

        # Dibujar caja del vehículo
        color = (0, 255, 0) if tipo.lower() == "person" else (255, 0, 0)
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        cv2.putText(frame, f"{tipo} ID:{track_id}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # Solo procesar coches
        if tipo.lower() == "car":
            roi_car = frame[y1:y2, x1:x2]
            if roi_car.size > 0:
                pred_matricula = modelo_matriculas.predict(roi_car, conf=0.3)
                matricula_roi = None
                placa_coords = None

                if pred_matricula:
                    for pm in pred_matricula:
                        if pm.boxes:
                            for bbox, conf_m in zip(pm.boxes.xyxy, pm.boxes.conf):
                                if conf_m > 0.4:
                                    mx1, my1, mx2, my2 = map(int, bbox)
                                    matricula_roi = roi_car[my1:my2, mx1:mx2]
                                    placa_coords = (mx1, my1, mx2, my2)
                                    break
                        if matricula_roi is not None:
                            break

                # OCR (EasyOCR + Tesseract)
                if matricula_roi is not None and placa_coords is not None:
                    mx1, my1, mx2, my2 = placa_coords

                    # --- PREPROCESAMIENTO ---
                    roi_gray = cv2.cvtColor(matricula_roi, cv2.COLOR_BGR2GRAY)
                    roi_gray = cv2.resize(roi_gray, None, fx=3, fy=3, interpolation=cv2.INTER_CUBIC)
                    roi_gray = cv2.GaussianBlur(roi_gray, (3, 3), 0)

                    # Binarización + inversión si es necesario
                    _, roi_thresh = cv2.threshold(roi_gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
                    if roi_thresh.mean() < 127:
                        roi_thresh = cv2.bitwise_not(roi_thresh)

                    # --- EASY-OCR ---
                    start_easy = time.time()
                    res_easy = reader_easy.readtext(roi_thresh)
                    end_easy = time.time()
                    if res_easy:
                        best_text = ""
                        best_conf = 0
                        for (bbox_e, text, conf_e) in res_easy:
                            if conf_e > best_conf and len(text) >= 4:
                                best_text = text
                                best_conf = conf_e
                        matricula_easy = ''.join([c for c in best_text.upper() if c.isalnum() or c == ' ']).strip()
                    time_easy = end_easy - start_easy

                    # --- TESSERACT ---
                    start_tess = time.time()
                    custom_config = r'--oem 3 --psm 8 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789 '
                    matricula_tesseract = pytesseract.image_to_string(
                        roi_thresh, config=custom_config
                    ).strip().upper()
                    matricula_tesseract = ''.join([c for c in matricula_tesseract if c.isalnum() or c == ' ']).strip()
                    end_tess = time.time()
                    time_tesseract = end_tess - start_tess

                    # --- ELECCIÓN DEL MEJOR RESULTADO ---
                    if len(matricula_tesseract) >= len(matricula_easy):
                        texto_final = matricula_tesseract
                    else:
                        texto_final = matricula_easy

                    # --- DIBUJAR MATRÍCULA ---
                    if texto_final:
                        cv2.rectangle(frame, (x1 + mx1, y1 + my1), (x1 + mx2, y1 + my2), (0, 0, 255), 2)
                        cv2.putText(frame, texto_final, (x1 + mx1, y1 + my1 - 5),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        # GUARDAR RESULTADO FRAME A FRAME
        resultados.append({
            "frame": frame_id,
            "tipo": tipo,
            "conf": conf,
            "id": track_id,
            "x1": x1, "y1": y1, "x2": x2, "y2": y2,
            "matricula_easy": matricula_easy,
            "matricula_tesseract": matricula_tesseract,
            "time_easy": time_easy,
            "time_tesseract": time_tesseract
        })

    # Escribir frame procesado en el video
    out.write(frame)
    frame_id += 1

# GUARDAR CSV Y FINALIZAR
df = pd.DataFrame(resultados)
df.to_csv(csv_output, index=False, sep=";")

cap.release()
out.release()
print("Procesamiento finalizado. Video y CSV generados correctamente.")


### Imagen

In [ ]:
import cv2
import pytesseract
import easyocr
import pandas as pd
from ultralytics import YOLO
import os

# Ruta al ejecutable de Tesseract
pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract.exe'

# Inicializar OCR (inglés → mejor para texto alfanumérico)
reader_easy = easyocr.Reader(['en'], gpu=True)

# Cargar modelos YOLO
modelo_vehiculos = YOLO("yolo11n.pt").to('cuda')
modelo_matriculas = YOLO("best.pt").to('cuda')

def procesar_imagen(image_path):
    # Obtener nombre y extensión de la imagen
    base_name = os.path.basename(image_path)
    name, ext = os.path.splitext(base_name)
    
    # Carpeta de salida = misma carpeta donde está el .py
    output_dir = os.getcwd()
    image_output = os.path.join(output_dir, f"{name}_resultado{ext}")
    csv_output = os.path.join(output_dir, f"{name}_resultado.csv")
    
    # Cargar imagen
    frame = cv2.imread(image_path)
    if frame is None:
        print(f"Error: no se puede abrir la imagen {image_path}")
        return
    
    resultados = []
    frame_id = 0

    # Detección y tracking de vehículos
    pred = modelo_vehiculos.track(frame, persist=True)[0]

    for obj in pred.boxes:
        x1, y1, x2, y2 = map(int, obj.xyxy[0])
        tipo = modelo_vehiculos.names[int(obj.cls[0])]
        conf = float(obj.conf[0])
        track_id = int(obj.id[0]) if hasattr(obj, 'id') else -1

        matricula_easy = ""
        matricula_tesseract = ""

        # Dibujar caja del vehículo
        color = (0, 255, 0) if tipo.lower() == "person" else (255, 0, 0)
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        cv2.putText(frame, f"{tipo} ID:{track_id}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # Solo procesar coches
        if tipo.lower() == "car":
            roi_car = frame[y1:y2, x1:x2]
            if roi_car.size > 0:
                pred_matricula = modelo_matriculas.predict(roi_car, conf=0.3)
                matricula_roi = None
                placa_coords = None

                if pred_matricula:
                    for pm in pred_matricula:
                        if pm.boxes:
                            for bbox, conf_m in zip(pm.boxes.xyxy, pm.boxes.conf):
                                if conf_m > 0.4:
                                    mx1, my1, mx2, my2 = map(int, bbox)
                                    matricula_roi = roi_car[my1:my2, mx1:mx2]
                                    placa_coords = (mx1, my1, mx2, my2)
                                    break
                        if matricula_roi is not None:
                            break

                # OCR (EasyOCR + Tesseract)
                if matricula_roi is not None and placa_coords is not None:
                    mx1, my1, mx2, my2 = placa_coords

                    # --- PREPROCESAMIENTO ---
                    roi_gray = cv2.cvtColor(matricula_roi, cv2.COLOR_BGR2GRAY)
                    roi_gray = cv2.resize(roi_gray, None, fx=3, fy=3, interpolation=cv2.INTER_CUBIC)
                    roi_gray = cv2.GaussianBlur(roi_gray, (3, 3), 0)
                    _, roi_thresh = cv2.threshold(roi_gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
                    if roi_thresh.mean() < 127:
                        roi_thresh = cv2.bitwise_not(roi_thresh)

                    # --- EASY-OCR ---
                    res_easy = reader_easy.readtext(roi_thresh)
                    if res_easy:
                        best_text = ""
                        best_conf = 0
                        for (bbox_e, text, conf_e) in res_easy:
                            if conf_e > best_conf and len(text) >= 4:
                                best_text = text
                                best_conf = conf_e
                        matricula_easy = ''.join([c for c in best_text.upper() if c.isalnum() or c == ' ']).strip()

                    # --- TESSERACT ---
                    custom_config = r'--oem 3 --psm 8 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789 '
                    matricula_tesseract = pytesseract.image_to_string(
                        roi_thresh, config=custom_config
                    ).strip().upper()
                    matricula_tesseract = ''.join([c for c in matricula_tesseract if c.isalnum() or c == ' ']).strip()

                    # --- ELECCIÓN DEL MEJOR RESULTADO ---
                    texto_final = matricula_tesseract if len(matricula_tesseract) >= len(matricula_easy) else matricula_easy

                    # --- DIBUJAR MATRÍCULA ---
                    if texto_final:
                        cv2.rectangle(frame, (x1 + mx1, y1 + my1), (x1 + mx2, y1 + my2), (0, 0, 255), 2)
                        cv2.putText(frame, texto_final, (x1 + mx1, y1 + my1 - 5),
                                    cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 2)

        # Guardar resultado
        resultados.append({
            "frame": frame_id,
            "tipo": tipo,
            "conf": conf,
            "id": track_id,
            "x1": x1, "y1": y1, "x2": x2, "y2": y2,
            "matricula_easy": matricula_easy,
            "matricula_tesseract": matricula_tesseract
        })

    # Guardar imagen y CSV en la carpeta del .py
    cv2.imwrite(image_output, frame)
    df = pd.DataFrame(resultados)
    df.to_csv(csv_output, index=False, sep=";")

    print(f"Procesamiento finalizado para {image_path}.")
    print(f"Imagen guardada como: {image_output}")
    print(f"CSV guardado como: {csv_output}")


In [ ]:
procesar_imagen("C:/Users/laura/OneDrive/Desktop/VC/Practica4/todo/images/4280KSW.jpg")
procesar_imagen("C:/Users/laura/OneDrive/Desktop/VC/Practica4/todo/images/0303BML_6225_aug5.jpg")
